In [ ]:
import pandas as pd
from ratdata import data_manager as dm
import re
import numpy as np

In [ ]:
dm.db_connect('rat_data.db')
update = False

In [ ]:
df = pd.read_excel('data/file overview.xlsx')

In [ ]:
files_not_in_db = []
for index, row in df.iterrows():
    fname = row['File name']
    if str(fname) != 'nan':
        if not re.match('.*\.mat', fname):
            fname = '.'.join([fname, 'mat'])
    stim = dm.StimSettings.select().join(dm.RecordingFile).where(dm.RecordingFile.filename == fname)
    if stim.count() == 0:
        files_not_in_db.append(fname)
    else:
        stim_setting = stim.get()
        xfa = str(row['DBS amplitude']).split('.')[0]
        dba = str(stim_setting.max_amplitude)
        if xfa != dba and (xfa != 'nan' or dba != 'None'):
                print('%s, ID%d: XLS %s DB %s' % (fname, stim_setting.stim_settings_id, xfa, dba))
                if xfa == 'nan' and update:
                    dm.StimSettings.update(max_amplitude=None).where(dm.StimSettings.stim_settings_id == stim_setting.stim_settings_id).execute()
                elif update:
                    dm.StimSettings.update(max_amplitude=int(xfa)).where(dm.StimSettings.stim_settings_id == stim_setting.stim_settings_id).execute()
print('FILES NOT IN DB:')
for s in files_not_in_db:
    print(s)